#  Multi class text classification using Google BERT and tensorflow

In [1]:
'''In this kernel we will implement multi class text classification using Google BERT and tensorflow'''
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import collections
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


['googleresearchbert', 'the-great-indian-data-scientist-hiring-challenge', 'bert-pretrained-models']


In [2]:
from __future__ import absolute_import
from __future__ import division

from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import numpy as np 
import pandas as pd
import re
import gc
print(os.listdir("../input"))
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report

%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings(action='once')
import pickle


from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime


import shutil

['googleresearchbert', 'the-great-indian-data-scientist-hiring-challenge', 'bert-pretrained-models']


Using TensorFlow backend.


In [3]:
!pip install bert-tensorflow


     |████████████████████████████████| 71kB 8.1MB/s 


In [4]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

--2019-07-03 03:13:32--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 2a00:1450:4013:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   107MB/s    in 3.6s    

2019-07-03 03:13:36 (107 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

--2019-07-03 03:13:37--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/

In [5]:
folder = 'model_folder'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [6]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [7]:
#import tokenization
#import modeling
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')
BERT_VOCAB= '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_config.json'

>> Model output directory: model_folder/outputs
>>  BERT pretrained directory: model_folder/uncased_L-12_H-768_A-12


In [8]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

## Data preprocessing and storing

In [9]:
import numpy as np                     # For mathematical calculations
import seaborn as sns                  # For data visualization
import matplotlib.pyplot as plt 
import seaborn as sn                   # For plotting graphs
import re
import nltk
%matplotlib inline
import warnings                        # To ignore any warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pandas import DataFrame
 
le = LabelEncoder()
 
df = pd.read_csv("../input/the-great-indian-data-scientist-hiring-challenge/Train.csv")

df['Item_Description'] = df['Item_Description'].apply(lambda x : re.sub(r'\b[A-Z]+\b', '',re.sub('\s+',' ',re.sub(r'[^a-zA-Z]', ' ',x))))
df['Item_Description'] = df['Item_Description'].apply(lambda x : x.lower())
identity_columns = ['CLASS-784', 'CLASS-95', 'CLASS-51', 'CLASS-559', 'CLASS-489', 'CLASS-913', 'CLASS-368', 'CLASS-816', 'CLASS-629', 'CLASS-177', 'CLASS-123', 'CLASS-671', 'CLASS-804', 'CLASS-453', 'CLASS-1042', 'CLASS-49', 'CLASS-947', 'CLASS-110', 'CLASS-278', 'CLASS-522', 'CLASS-606', 'CLASS-651', 'CLASS-765', 'CLASS-953', 'CLASS-839', 'CLASS-668', 'CLASS-758', 'CLASS-942', 'CLASS-764', 'CLASS-50', 'CLASS-75', 'CLASS-74', 'CLASS-783', 'CLASS-323', 'CLASS-322', 'CLASS-720', 'CLASS-230', 'CLASS-571'] 
for key in identity_columns:
    df.Product_Category[df.Product_Category==key] = identity_columns.index(key)+1
print(len(identity_columns))

df2 = pd.DataFrame({'text':df['Item_Description'].replace(r'\n',' ',regex=True),
            'label':LabelEncoder().fit_transform(df['Product_Category'].replace(r' ','',regex=True)),
            })

# Creating train and val dataframes according to BERT
X_train, X_test, y_train, y_test = train_test_split(df2["text"].values, df2["label"].values, test_size=0.2, random_state=42)
X_train, y_train = df2["text"].values, df2["label"].values



 
# Creating test dataframe according to BERT
testpd = pd.read_csv("../input/the-great-indian-data-scientist-hiring-challenge/Test.csv")
testpd['Item_Description'] = testpd['Item_Description'].apply(lambda x : re.sub(r'\b[A-Z]+\b', '',re.sub('\s+',' ',re.sub(r'[^a-zA-Z]', ' ',x))))
testpd['Item_Description'] = testpd['Item_Description'].apply(lambda x : x.lower())
test = pd.DataFrame({'text':testpd['Item_Description'].replace(r'\n',' ',regex=True)})
test = test["text"].values

# Saving dataframes to .tsv format as required by BERT
#X_train.to_csv('train.tsv', sep='\t', index=False, header=False)
#X_test.to_csv('dev.tsv', sep='\t', index=False, header=False)
#test.to_csv('test.tsv', sep='\t', index=False, header=False)

<>:19: DeprecationWarning: invalid escape sequence \s


38


In [10]:
X_train[:5] # check training data

array(['   apr store management real estate real estate services real estate general search appraisal realtor commission ',
       '   nov store management real estate real estate services real estate general search appraisal realtor commission ',
       '  nov store management real estate real estate services real estate general search appraisal realtor commission ',
       '     aug store management real estate real estate services real estate general search appraisal realtor commission ',
       '     mar store management real estate real estate services real estate general search appraisal realtor commission '],
      dtype=object)

## Initialize BERT model

In [11]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid=f'{set_type}'
    examples = []
    if set_type == 'train':
        for line, label in zip(lines, labels):
            
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 100
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(38)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Train the model using traing data

In [12]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.now()))

Please wait...
>> Started training at 2019-07-03 03:13:54.365906 
  Num examples = 5719
  Batch size = 16
>> Finished training at 2019-07-03 03:19:15.209332


In [13]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 500

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

## Test model on validation dataset

In [14]:
predict_examples = create_examples(X_test, 'test')
predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

In [15]:
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

{}


In [16]:
print(preds)

[2, 3, 30, 3, 3, 33, 29, 28, 33, 30, 3, 30, 30, 3, 33, 9, 8, 9, 30, 9, 3, 3, 18, 9, 3, 12, 30, 3, 33, 3, 3, 8, 3, 5, 2, 28, 3, 30, 3, 19, 3, 30, 30, 33, 21, 6, 33, 18, 33, 3, 33, 8, 30, 3, 3, 30, 30, 30, 33, 29, 3, 28, 3, 3, 3, 28, 28, 19, 2, 29, 30, 28, 3, 9, 9, 33, 30, 3, 30, 3, 30, 3, 30, 33, 3, 33, 2, 30, 3, 30, 27, 33, 30, 3, 30, 28, 27, 3, 12, 9, 3, 3, 3, 33, 33, 9, 33, 3, 33, 9, 28, 2, 3, 35, 3, 30, 30, 3, 30, 3, 33, 33, 9, 30, 28, 35, 9, 33, 33, 21, 3, 9, 33, 30, 19, 19, 30, 30, 2, 8, 30, 30, 28, 3, 27, 3, 29, 29, 33, 9, 28, 18, 33, 2, 3, 3, 3, 18, 9, 6, 2, 21, 6, 33, 33, 30, 30, 28, 3, 3, 33, 33, 6, 29, 9, 30, 3, 18, 28, 2, 2, 3, 19, 2, 21, 19, 2, 3, 9, 30, 3, 30, 30, 30, 35, 19, 33, 9, 28, 30, 14, 29, 6, 3, 28, 8, 3, 33, 2, 29, 33, 30, 29, 27, 33, 29, 9, 2, 29, 5, 3, 3, 30, 19, 27, 3, 27, 3, 29, 29, 3, 29, 3, 3, 3, 30, 19, 19, 29, 30, 3, 19, 29, 3, 12, 3, 33, 6, 3, 29, 30, 9, 2, 9, 3, 30, 30, 6, 28, 28, 8, 29, 28, 33, 28, 5, 33, 30, 30, 3, 3, 3, 12, 3, 3, 2, 29, 3, 33, 30, 3,

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
print("Accuracy of BERT is:",accuracy_score(y_test,preds))

Accuracy of BERT is: 0.9012237762237763


Accuracy obtained is about 90.12%

In [19]:
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         8
           2       1.00      1.00      1.00        84
           3       1.00      1.00      1.00       302
           4       0.00      0.00      0.00         1
           5       1.00      1.00      1.00        10
           6       0.53      1.00      0.69        18
           7       0.00      0.00      0.00         1
           8       1.00      1.00      1.00        21
           9       1.00      1.00      1.00        82
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00        11
          12       1.00      1.00      1.00         8
          13       0.00      0.00      0.00         5
          14       0.50      0.11      0.18         9
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         1
          17       0.00    

## Preprocess actual test data

In [20]:
testpd = pd.read_csv("../input/the-great-indian-data-scientist-hiring-challenge/Test.csv")
testpd['Item_Description'] = testpd['Item_Description'].apply(lambda x : re.sub(r'\b[A-Z]+\b', '',re.sub('\s+',' ',re.sub(r'[^a-zA-Z]', ' ',x))))
testpd['Item_Description'] = testpd['Item_Description'].apply(lambda x : x.lower())
testln = pd.DataFrame({'guid':testpd['Inv_Id'],
    'text':testpd['Item_Description'].replace(r'\n',' ',regex=True)})
testl = testln["text"].values

## Run predictions on actual test data

In [21]:
predict_test = create_examples(testl, 'test')
predict_features1 = run_classifier.convert_examples_to_features(
    predict_test, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn1 = input_fn_builder(
    features=predict_features1,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result1 = estimator.predict(input_fn=predict_input_fn1)

In [22]:
preds1 = []
for prediction in result1 :
    #print(prediction)
    preds1.append(np.argmax(prediction['probabilities']))

{}


In [23]:
print(preds1)

[19, 19, 19, 19, 19, 19, 19, 29, 29, 5, 6, 14, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 6, 19, 19, 19, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 6, 6, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 6, 6, 6, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 12, 12, 12, 12, 29, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 12, 12, 12, 12, 12, 12, 14, 29, 29, 6, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 5, 19, 19, 19, 19, 19, 19,

In [24]:
def create_output(predictions):
    probabilities = []
    for (i, prediction) in enumerate(predictions):
        preds = prediction
        probabilities.append(preds)
        #print(preds)
    dff = pd.DataFrame(probabilities)
    dff.head()
    
    #dff.columns = identity_columns
    
    return dff

In [25]:
#output_df = create_output(preds1)
predslab=[identity_columns[x] for x in preds1]
testln["Product_Category"]=predslab
merged_df =  testln
submission = merged_df.drop(['text'], axis=1)
submission.to_csv("sample_submission1.csv", index=False)

In [26]:
submission.head()


,guid,Product_Category
0,6,CLASS-522
1,12,CLASS-522
2,14,CLASS-522
3,18,CLASS-522
4,19,CLASS-522


With this we have completed predicting product class and stored the results to a csv file.
Hope you enjoyed the kernel.
Feel free to use this kernel as a base doing your own multi class text classification.